## Load environment and check for folder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import subprocess
import os
base_dir = "/content/gdrive/MyDrive/final-project-paralelizacao-tecnica-de-meios-tons"
get_ipython().run_line_magic('cd', base_dir)

# assert that we have the exercise folder
assert 'repositorio' in os.listdir(), "Exercise folder not found"

Cuda de Técnicas de de Meios-Tons

In [ ]:
import os
import subprocess

# Diretórios principais
base_dir = os.getcwd()
exercise_dir = os.path.join(base_dir, "repositorio")
build_dir = os.path.join(exercise_dir, "build")
input_base_dir = os.path.join(base_dir, "img", "ppm")
output_base_dir = os.path.join(exercise_dir, "out")

# Função para executar comandos shell
def run_command(command, work_dir):
    return subprocess.run(
        command, shell=True, cwd=work_dir,
        text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )

# Preparar build: limpar e criar pasta build/
print("🔧 Limpando diretório de build...")
run_command("cmake -E remove_directory build", exercise_dir)
run_command("cmake -E make_directory build", exercise_dir)

# Executar CMake e verificar erro
print("⚙️ Executando CMake...")
cmake_result = run_command("cmake ..", build_dir)
print("CMAKE STDOUT:\n", cmake_result.stdout)
print("CMAKE STDERR:\n", cmake_result.stderr)
assert cmake_result.returncode == 0, "Erro na configuração com CMake"

# Ajustar input_base_dir para ser relativo ao exercise_dir
input_base_dir = os.path.join(exercise_dir, "..", "img", "ppm")
input_base_dir = os.path.abspath(input_base_dir)
print("🧱 Compilando com make...")
make_result = run_command("make -j", build_dir)
print("MAKE STDOUT:\n", make_result.stdout)
print("MAKE STDERR:\n", make_result.stderr)
assert make_result.returncode == 0, "Erro na compilação (make)"

# Encontrar todas as imagens ppm no dataset img/ppm e subpastas
test_images = []
for root, dirs, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".ppm"):
            full_path = os.path.join(root, file)
            # Caminho relativo em relação a input_base_dir
            rel_path = os.path.relpath(full_path, input_base_dir)
            test_images.append(rel_path)

print(f"🖼️ Imagens encontradas: {len(test_images)}")

exec_path = os.path.join(build_dir, "fsh_cuda")
subprocess.run(f"chmod +x {exec_path}", shell=True)

# Parâmetros do programa
p = 0.5  # parâmetro de ruído (default)
stochastic = 1  # usar dithering estocástico (default)
grayscale_flag = "-g"  # converter para grayscale

# Executar para todas imagens
for i, rel_image_path in enumerate(test_images, start=1):
    print(f"\n===== 🧪 Teste {i} - Imagem: {rel_image_path} =====")

    img_path = os.path.join(input_base_dir, rel_image_path)
    
    # Preparar caminho de saída mantendo subpastos
    output_path = os.path.join(output_base_dir, rel_image_path)
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)

    # Ajustar nome do arquivo de saída
    if stochastic:
        output_path = output_path.replace(".ppm", "_stochastic.ppm")
    else:
        output_path = output_path.replace(".ppm", ".ppm")

    # Montar comando
    command = f"{exec_path} {img_path} {output_path} {p} {stochastic} {grayscale_flag}"
    print(f"▶️ Executando: {command}")

    run = subprocess.run(
        command, shell=True, cwd=exercise_dir,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True
    )

    # Mostrar saída
    print("\n=== 📤 Saída do programa (stdout) ===")
    print(run.stdout)

    print("\n=== 🐞 Logs do programa (stderr) ===")
    print(run.stderr)

    # Exibir tempo de execução se existir
    time_file = os.path.join(build_dir, f"time_{i}.txt")
    if os.path.exists(time_file):
        with open(time_file) as f:
            runtime = f.read().strip()
        print(f"\n⏱ Tempo de execução: {runtime} segundos")
    else:
        print(f"\n⚠️ Falha ao medir tempo para {rel_image_path}")
